In [1]:
import os
import pandas as pd

all_data = pd.DataFrame()
base_data_dir = "methylation_VAE_TRAIN_DATA/TCGA-COAD"
#base_data_dir = "cases"

print(os.listdir("cases"))

illumina_450k_count = 0
illumina_27k_count = 0
total_files = 0

for case_folder in os.listdir(base_data_dir):
    case_folder_path = os.path.join(base_data_dir, case_folder)
    
    if os.path.isdir(case_folder_path):
        dnamethylation_folder = os.path.join(case_folder_path, "DNA Methylation")
        
        if os.path.exists(dnamethylation_folder):
            for file_name in os.listdir(dnamethylation_folder):
                if file_name.endswith(".txt"):
                    beta_file_path = os.path.join(dnamethylation_folder, file_name)
                    beta_data = pd.read_csv(beta_file_path, sep="\t", header=0, index_col=0)
                    beta_data = beta_data.transpose()

                    print(f"Processing file: {beta_file_path}")
                    print("beta_data shape before processing:", beta_data.shape)

                    if beta_data.shape[1] >= 30000 and beta_data.shape[1] <= 500000:
                        illumina_450k_count += 1
                    else:
                        illumina_27k_count += 1
                        total_files += 1
                        continue

                    existing_cpgs = set(all_data.index)
                    new_cpgs = set(beta_data.index)

                    new_data = beta_data.loc[beta_data.index.difference(existing_cpgs)]

                    if not new_data.empty:
                        all_data = pd.concat([all_data, new_data], axis=0, join='outer')

                    print("all_data shape after processing:", all_data.shape)

                    total_files += 1

illumina_450k_percentage = (illumina_450k_count / total_files) * 100 if total_files else 0
illumina_27k_percentage = (illumina_27k_count / total_files) * 100 if total_files else 0

print(f"Illumina 450k count: {illumina_450k_count}, Percentage: {illumina_450k_percentage:.2f}%")
print(f"Illumina 27k count: {illumina_27k_count}, Percentage: {illumina_27k_percentage:.2f}%")
print("Final shape of all_data:", all_data.shape)

['b3c267aa-345d-40d8-a8da-f9f08c64f9c7', 'a85e8208-a6f5-4e91-8f35-ef574f4273fc', '835c3e23-ac2b-4bc9-ac65-e3e81adecf77', 'ea923671-3903-4a79-830d-e7e4b1eabab0', '131e0571-de86-415f-b3fb-dc9f971258d3', '1c111912-c664-42f5-9802-065b1994a2f4', 'acd51808-870c-4684-a2de-aa0b058729e1', 'a9cb21d1-4c4a-4f30-ad4d-c54bed26f187', '912b61a1-6a5b-451c-b753-9c3dac03e8cb', '14efca0c-882d-4fba-b15c-64af6e213b80', '07dc5a1d-64b7-42ba-928e-9753bd533c1e', '3e1886a8-2ed2-41ee-8b58-10f5321ade6f', 'cbe23011-93b7-4e9f-b934-bd0122f91f54', '1136dd50-242a-4659-aad4-c53f9e759bb3', '563d67a1-f1ba-4722-aa2d-9b35b1689468', '4f601d7b-8db1-4c6d-9374-21dcd804980d', 'e3b0ee77-c419-4b2b-833f-90a221415489', '622cc1c4-d721-4b5c-a88b-216d6368660b', 'a782f174-241d-42c6-9bc4-61cc39f8b397', '8c0894fb-5e78-40ae-be0b-329ca30f758a', 'dc22eba5-6fb7-4875-b30a-99fa00758301', '0a815133-0edd-4422-89f2-2f067b6f2156', '17633a60-e12b-4a7c-bd8a-c16341ddc606', '9dcb4e16-b582-4cd0-87cf-04f480ec2cec', '5eed7e8a-c219-4497-a99f-bfc48faca4d8',

FileNotFoundError: [Errno 2] No such file or directory: 'methylation_VAE_TRAIN_DATA/TCGA-COAD'

Final shape of all_data: (352, 486426)

In [ ]:
#all_data.to_csv("final_methylation_data_TCGA-READ.csv")

all_data.to_parquet("aggregated_methylation_data/final_methylation_data_TCGA-LUAD.parquet", index=False)

In [ ]:
import rpy2.robjects as ro
import pandas as pd

custom_path = "/aggregated_methylation_data"

ro.r(f'library(IlluminaHumanMethylation450kanno.ilmn12.hg19, lib.loc="{custom_path}")')
ro.r('data(IlluminaHumanMethylation450kanno.ilmn12.hg19)')
annotation_data = ro.r('IlluminaHumanMethylation450kanno.ilmn12.hg19')

print(annotation_data)